In [1]:
import numpy as np
from scipy.stats import binom

In [7]:
# Parameters 

h = 1/7
pho = 2/3
gamma = 2/3
R = 19
T = 7*R

In [8]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [9]:
# Mutation transition

def mutation(theta, betas, S, E, I, R):
    #
    M = theta[:16].reshape(4,4)
    betas = 1/n * np.dot(M, I)
    B = np.random.binomial(S, np.ones(4) - np.exp(-h*betas)) 
    C = np.random.binomial(E, 1 - np.exp(-h*pho)) 
    D = np.random.binomial(I, 1 - np.exp(-h*gamma)) 
    S_new = S - B
    E_new = E + B - C
    I_new = I + C - D
    R_new = R - D
    return S_new, E_new, I_new, R_new

In [10]:
# Likelihood of y knowing theta

def likelihood(y, theta, betas, S, E, I, R):
    C = np.zeros(shape=(7, 4))
    S_new, E_new, I_new, R_new =  S, E, I, R
    for i in range(7):
        S_new, E_new, I_new, R_new = mutation(theta, betas, S_new, E_new, I_new, R_new)
        C[i] = np.random.binomial(E_new, 1 - np.exp(-h*pho))
    return(binom.pmf(y, np.sum(C, axis=0), theta[16:]))

In [ ]:
# Initial settings
S_init = np.array([948, 1689, 3466, 1894])
E_init = np.array([0, 0, 0, 0])
I_init = np.array([1, 1, 1, 1])
R_init = np.array([0, 0, 0, 0])

def BootstrapFilter(ESS_min, N, R=19):
    thetas = []
    for n in range(N):
        thetas.append(np.random.gamma(5, 1, 16)+